We have chosen SQLite for implementation of the sample demonstration because of its excellent performance as it stores data in-memory. The database is stored in the disk under the name "sample_225_database.db". The entire database which includes all the tables and the views stored on disk thus enabling shorter data access time. 


The script below is used to create, insert and query the sample_redcap_data table. The sample_redcap_data table is the heart of the implementation as it contains all the data associated with the project. For demonstration purposes, we have incorporated the below functionality for a single project with a project_id of 225. But the implementation is designed to be extensible for a wide variety of databases that reside in REDCap. We have included 3 events for demonstration to illustrate how events are managed. Another column named record has a one-to-one mapping with patient_id wherein the column patient_id is included in another table named sample_patient_id. We are utilizing the python library named sqlite3 which has proved to be versatile in connecting to SQLite backend from python and also has excellent query performance.

We have created two functions namely "create_table_data()" and "create_inserts_data()". The function "create_table_data()" is used to create the table named "sample_redcap_data" and the function "create_inserts_data()" is used to insert the data. The data to be inserted is read from a text file named "sample_redcap_data.txt" the data stored in the text file is evaluated as a python expression as shown in the code where it is read in as a tuple. We are calling the function named "executemany()" to perform all the inserts at one go. 

To ensure that all data was inserted successfully, we also execute the select query to verify data insertion.


In [1]:
import sqlite3
import os


#Connect to SQLite database instance named sample_225_database.db
conn = sqlite3.connect(os.path.expanduser('~') + "\\sample_225_database.db")
c = conn.cursor()


def create_table_data():
    #Query to create table sample_redcap_data
    c.execute('Create table if not exists sample_redcap_data(project_id INTEGER, event_id INTEGER, record INTEGER,field_name text,value text, instance text)')

#Read text file containing tuple of data to be inserted
readinsertdata = open(os.path.expanduser('~') + '\\sample_redcap_data.txt')

#Evaluate the tuple present in the text file as a python expression
readdata = eval(readinsertdata.read())

def create_inserts_data():
    
    #Query to insert data into sample_redcap_data table
    c.executemany('insert into sample_redcap_data(project_id, event_id,record,field_name,value,instance) values(?,?,?,?,?,?)',readdata)
    conn.commit()
    
    # Print the table sample_redcap_data
    c.execute('''SELECT * FROM sample_redcap_data''')
    for row in c:
        print(row)
 
    conn.close()
    c.close()
    conn.close()
    
create_table_data()
create_inserts_data()

(225, u'bmi_location_form', u'age', u'1', u'>10', u'radio')
(225, u'bmi_location_form', u'age', u'2', u'10-30', u'radio')
(225, u'bmi_location_form', u'age', u'3', u'40-60', u'radio')
(225, u'bmi_location_form', u'age', u'60+', u'60+', u'radio')
(225, u'demographics_form', u'alcohol_amount', u'1', u'<100 oz', u'checkbox')
(225, u'demographics_form', u'alcohol_amount', u'2', u'100-200oz', u'checkbox')
(225, u'demographics_form', u'alcohol_amount', u'3', u'>200oz', u'checkbox')
(225, u'drug_form', u'beta-blockers', u' 2', u'1-2 years', u'checkbox')
(225, u'drug_form', u'beta-blockers', u' 3', u'>2years', u'checkbox')
(225, u'drug_form', u'beta-blockers', u'1', u'>1 year', u'checkbox')
(225, u'comorbidities_form', u'cholesterol', u'1', u'Yes', u'radio')
(225, u'comorbidities_form', u'cholesterol', u'2', u'No', u'radio')
(225, u'bmi_location_form', u'city', u' 1', u'Buffalo', u'radio')
(225, u'bmi_location_form', u'city', u'2', u'Other', u'radio')
(225, u'bmi_location_form', u'county', u'1

The code snippet below is used to create, insert and select the data from the sample_redcap_metadata table. 
The table contains the form name each field is associated with along with the field_type which is represented by the 
Column named element_type. The fields can be of 3 types namely select, radio and checkbox. The column element_enum contains 
all the descriptions associated with every corresponding field and it uses '\n' as the delimiter. 

This table is mainly used for joining with the sample_redcap_data table to fetch the form names associated with every field. Although the table contains descriptions for every field, the structure of the descriptions makes it unviable to join and fetch descriptions. In order to overcome this issue, we have created another table i.e a lookup table named "gt_lookup_sample_data".  Details about this table are present below.



In [ ]:
import sqlite3
import os

#Connect to SQLite database instance named sample_225_database.db
conn = sqlite3.connect(os.path.expanduser('~') + "\\sample_225_database.db")
c = conn.cursor()


def create_table_metadata():
    
    #Create table sample_redcap_metadata
    c.execute('Create table if not exists sample_redcap_metadata(project_id INTEGER, field_name varchar(100), form_name varchar(100),element_type varchar(200),element_enum varchar(200))')

#Read text file named sample_redcap_metadata.txt 
readinsertmeta = open(os.path.expanduser('~') + '\\sample_redcap_metadata.txt')

#Evaluate the tuple being read as a dictionary
readmeta = eval(readinsertmeta.read())

def create_inserts_metadata():
    
    #Insert data into sample_redcap_metadata table
    c.executemany('insert into sample_redcap_metadata(project_id, field_name,form_name,element_type,element_enum) values(?,?,?,?,?)',readmeta)
    conn.commit()
    
    # Print the contents of sample_redcap_metadata table
    c.execute('''SELECT * FROM sample_redcap_metadata''')
    for row in c:
        print(row)
 
    conn.close()
   
   
    
create_table_metadata()
create_inserts_metadata()

The script contains the creation, insertion and selection for the table named gt_lookup_sample_data. The primary use of this table is to fetch the descriptions of the codes present in the sample_redcap_data table. The descriptions present in the sample_redcap_metadata table are broken down using '\n' as the delimiter such that each record has a separate description associated with every field. 

Viewing data in such a manner is desired while querying data for answering scientific research questions. The descriptions 
present in this table are fetched in the attribute tables by joining on this table thus making queries very intuitive. 
This table is essentially resembles the sample_redcap_metadata table with the main distinction being the descriptions split with a '\n' character.


In [1]:
import sqlite3
import os

#Connect to database named sample_225_database.db
conn = sqlite3.connect(os.path.expanduser('~') + "\\sample_225_database.db")
c = conn.cursor()


#Create the lookup table
def create_table_lookup():
    c.execute('Create table if not exists gt_lookup_sample_data(project_id INTEGER, form_name varchar(100), field_name varchar(100),code varchar(100), label varchar(200), element_type varchar(200))')

#Read data from the tuple created in text file
readinserts = open(os.path.expanduser('~') + '\\gt_lookup_sample_data.txt')


#Evaluate the tuple being read as a python expression
readfile = eval(readinserts.read())

def create_inserts_lookup():
    
    #Insert data into the lookup table
    c.executemany('insert into gt_lookup_sample_data(project_id, form_name, field_name, code, label, element_type) values(?,?,?,?,?,?)',readfile)
    conn.commit()
    
    # Print the contents of lookup table
    c.execute('''SELECT * FROM gt_lookup_sample_data''')
    for row in c:
        print(row)
 
   
    c.close()
    conn.close()
    
create_table_lookup()
create_inserts_lookup()

(225, u'bmi_location_form', u'age', u'1', u'>10', u'radio')
(225, u'bmi_location_form', u'age', u'2', u'10-30', u'radio')
(225, u'bmi_location_form', u'age', u'3', u'40-60', u'radio')
(225, u'bmi_location_form', u'age', u'60+', u'60+', u'radio')
(225, u'demographics_form', u'alcohol_amount', u'1', u'<100 oz', u'checkbox')
(225, u'demographics_form', u'alcohol_amount', u'2', u'100-200oz', u'checkbox')
(225, u'demographics_form', u'alcohol_amount', u'3', u'>200oz', u'checkbox')
(225, u'drug_form', u'beta-blockers', u' 2', u'1-2 years', u'checkbox')
(225, u'drug_form', u'beta-blockers', u' 3', u'>2years', u'checkbox')
(225, u'drug_form', u'beta-blockers', u'1', u'>1 year', u'checkbox')
(225, u'comorbidities_form', u'cholesterol', u'1', u'Yes', u'radio')
(225, u'comorbidities_form', u'cholesterol', u'2', u'No', u'radio')
(225, u'bmi_location_form', u'city', u' 1', u'Buffalo', u'radio')
(225, u'bmi_location_form', u'city', u'2', u'Other', u'radio')
(225, u'bmi_location_form', u'county', u'1

The script below is used to create the attribute tables by field names. The base table sample_redcap_data table is split into separate attribute tables such that an attribute table for every field name is created. The script is used to generate queries to create the attribute tables. The output of the script is a bunch of queries used to create the attribute tables.

The attribute tables are created for both checkbox and non-checkbox fields. We have made a distinction between checkbox and non-checkbox fields at this point so as to make it easier while creating the views. The views use a "group_concat()" function which will be addressed later. The script reads data from two separate csv files where one of the csv files contains the data associated with only checkbox fields and the other contains data associated with only non-checkbox fields. 

The script appends the keyword "checkbox_table" for all checkbox fields to provide a clear distinction and the non-checkbox fields' tables start with the string "table_". The output of this script is to be executed on the preferred SQLite environment (SQLIte command prompt, SQLite Studio, DBVisualizer, etc.). On execution, all the desired attribute tables by field names are generated.

The main advantage of attribute tables is that they help increase query performance rapidly as they are small in nature and also they play a pivotal role for creating the views based on form names. The attribute tables associated with a single form are all joined "across" to create the final views by form names. More details about the creation of views are explained in the subsequent sections.



In [27]:
import sqlite3
import pandas as pd
import getpass

import os



project_id = raw_input("REDCap Project ID#: ")

#function to create views
def create_att_table_sql(project_id):

    #read csv into pandas daraframe
    reqpd = pd.read_csv(os.path.expanduser('~') + '\\sample_data_tables.csv')
    ps = reqpd['field_name'].tolist()


    view_text = ""

    # Fetching fields that have checkboxes
    with open(os.path.expanduser('~') + '\\Samp_dict_cb.txt') as checkboxele:

        checkboxlist = checkboxele.readlines()
        checkboxstr = ",".join(checkboxlist)
        checkstr = checkboxstr.split(",")
        cblist = [x.strip("'") for x in checkstr]

        # Typecasting to set to remove duplicate checkbox fields
        checkboxset = set(cblist)

    # this returns fields that are not checkboxes
    listoffields = list(set(ps).difference(set(cblist)))


    # Iterating through the set of checkbox fields
    # Then, generating a SQL script that creates attribute tables
    # for only those containing checkboxes
    # The string "view_text" returns the query that is used to generate the desired attribute tables.

    for cb in set(checkboxset):
        view_text +=  """\n\n\nDROP TABLE IF EXISTS checkbox_table_""" + str(project_id) + "_" + cb + ";" 
        view_text += """\n\n\nCREATE table checkbox_table_""" + str(project_id) +"""_""" + cb + \
               """ as SELECT d.event_id,d.project_id, d.record, p.patient_id""" + \
                    ",g.form_name,d.field_name,d.value,gt.label,d.instance"""\
        "\nFROM sample_redcap_data d " \
                 "\ninner join sample_redcap_metadata g  " \
        "on g.field_name = d.field_name \n and d.project_id = g.project_id" \
        "\ninner join gt_lookup_sample_data gt on  gt.field_name = d.field_name \n and gt.code = d.value"  \
        "\ninner join sample_patient_id p on p.record = d.record"\
        "\nWHERE d.field_name = " + "'" + cb + "'" +\
                     " \norder by d.record;"


    #get unique fiels_name
    #used for passsing dynamically to the query used for creating views
    setps = set(ps)


    for nocb in listoffields:

        #query to create views dynamically - field_names are dynamically fetched from previous stop
        #The query ensures that rows are not repeated thus preventing data duplication
        #this query logic handles repeating forms
        view_text += """\n\n\nDROP TABLE IF EXISTS table_""" + str(project_id) + "_" + nocb + ";" + \
        """\n\n\nCREATE table table_""" + str(project_id) +"""_""" + nocb + \
              """ as SELECT d.event_id,d.project_id, d.record, p.patient_id""" + \
                    ",g.form_name,d.field_name,d.value,gt.label,d.instance"""\
        "\nFROM sample_redcap_data d " \
                 "\ninner join sample_redcap_metadata g  " \
        "on g.field_name = d.field_name \n and d.project_id = g.project_id" \
        "\ninner join gt_lookup_sample_data gt on  gt.field_name = d.field_name \n and gt.code = d.value"  \
        "\ninner join sample_patient_id p on p.record = d.record"\
        "\nWHERE d.field_name = " + "'" + nocb + "'" +\
                     " \norder by d.record;"

    print view_text
  

create_att_table_sql(225)

REDCap Project ID#: 225



DROP TABLE IF EXISTS checkbox_table_225_beta-blockers;


CREATE table checkbox_table_225_beta-blockers as SELECT d.event_id,d.project_id, d.record, p.patient_id,g.form_name,d.field_name,d.value,gt.label,d.instance
FROM sample_redcap_data d 
inner join sample_redcap_metadata g  on g.field_name = d.field_name 
 and d.project_id = g.project_id
inner join gt_lookup_sample_data gt on  gt.field_name = d.field_name 
 and gt.code = d.value
inner join sample_patient_id p on p.record = d.record
WHERE d.field_name = 'beta-blockers' 
order by d.record;


DROP TABLE IF EXISTS checkbox_table_225_tobacco_amount;


CREATE table checkbox_table_225_tobacco_amount as SELECT d.event_id,d.project_id, d.record, p.patient_id,g.form_name,d.field_name,d.value,gt.label,d.instance
FROM sample_redcap_data d 
inner join sample_redcap_metadata g  on g.field_name = d.field_name 
 and d.project_id = g.project_id
inner join gt_lookup_sample_data gt on  gt.field_name = d.field_name 
 and gt.c

The script below is used to fetch the patient id's for every patient. We create, insert and select rows as shown in the script below. Each patient id has a one-to-one mapping with the record column present in the sample_redcap_table and all the attribute tables. In order to fetch the patient id, we join on the table created below.

In [1]:
import sqlite3
import os

#Connect to the SQLite database
conn = sqlite3.connect(os.path.expanduser('~') + "\\sample_225_database.db")
c = conn.cursor()


def create_table_patient_id():
    
    #Create table sample_patient_id containing record and associated patient_id
    c.execute('Create table if not exists sample_patient_id(event_id INTEGER, project_id INTEGER, record INTEGER,field_name text,patient_id INTEGER)')

#Read the tuple of patient_id's
readinsertdata = open(os.path.expanduser('~') + '\\sample_patient_id.txt')

#Evaluate the tuple as a python expression
readdata = eval(readinsertdata.read())

def create_inserts_patient_id():
    #Insert data into sample_patient_id table
    c.executemany('insert into sample_patient_id(event_id,project_id,record,field_name,patient_id) values(?,?,?,?,?)',readdata)
    conn.commit()
    
    # Print the contaents of sample_patient_id table
    c.execute('''SELECT * FROM sample_patient_id''')
    for row in c:
        print(row)
 
    conn.close()
    c.close()
    conn.close()
    
create_table_patient_id()
create_inserts_patient_id()

(227, 225, 88, u'medrecno', 1487)
(226, 225, 49, u'medrecno', 1715)
(228, 225, 39, u'medrecno', 1468)
(227, 225, 8, u'medrecno', 1489)
(227, 225, 57, u'medrecno', 1049)
(226, 225, 33, u'medrecno', 1186)
(226, 225, 51, u'medrecno', 1397)
(227, 225, 60, u'medrecno', 1913)
(227, 225, 64, u'medrecno', 1476)
(226, 225, 89, u'medrecno', 1104)
(228, 225, 27, u'medrecno', 1412)
(226, 225, 4, u'medrecno', 1766)
(228, 225, 12, u'medrecno', 1061)
(227, 225, 51, u'medrecno', 1902)
(226, 225, 63, u'medrecno', 1268)
(228, 225, 76, u'medrecno', 1571)
(228, 225, 71, u'medrecno', 1248)
(228, 225, 34, u'medrecno', 1082)
(226, 225, 57, u'medrecno', 1798)
(227, 225, 53, u'medrecno', 1636)
(228, 225, 64, u'medrecno', 1445)
(227, 225, 27, u'medrecno', 1342)
(226, 225, 62, u'medrecno', 1168)
(227, 225, 82, u'medrecno', 1798)
(226, 225, 67, u'medrecno', 1748)
(226, 225, 100, u'medrecno', 1646)
(226, 225, 52, u'medrecno', 1177)
(226, 225, 10, u'medrecno', 1016)
(227, 225, 89, u'medrecno', 1976)
(227, 225, 88, 

(228, 225, 21, u'medrecno', 1238)
(226, 225, 4, u'medrecno', 1211)
(228, 225, 14, u'medrecno', 1219)
(228, 225, 64, u'medrecno', 1603)
(226, 225, 87, u'medrecno', 1148)
(227, 225, 40, u'medrecno', 1393)
(228, 225, 24, u'medrecno', 1923)
(227, 225, 89, u'medrecno', 1558)
(227, 225, 11, u'medrecno', 1020)
(228, 225, 18, u'medrecno', 1512)
(228, 225, 2, u'medrecno', 1158)
(228, 225, 3, u'medrecno', 1626)
(228, 225, 87, u'medrecno', 1163)
(226, 225, 64, u'medrecno', 1590)
(227, 225, 62, u'medrecno', 1208)
(226, 225, 59, u'medrecno', 1984)
(227, 225, 4, u'medrecno', 1282)
(227, 225, 29, u'medrecno', 1561)
(226, 225, 99, u'medrecno', 2001)
(226, 225, 90, u'medrecno', 1509)
(228, 225, 12, u'medrecno', 1390)
(228, 225, 55, u'medrecno', 1433)
(227, 225, 46, u'medrecno', 1487)
(226, 225, 37, u'medrecno', 1568)
(226, 225, 10, u'medrecno', 1216)
(228, 225, 39, u'medrecno', 1665)
(228, 225, 27, u'medrecno', 1191)
(228, 225, 68, u'medrecno', 1084)
(227, 225, 31, u'medrecno', 1447)
(228, 225, 63, u'm

(227, 225, 17, u'medrecno', 1007)
(228, 225, 82, u'medrecno', 1706)
(226, 225, 27, u'medrecno', 1748)
(227, 225, 91, u'medrecno', 1127)
(226, 225, 27, u'medrecno', 1981)
(227, 225, 30, u'medrecno', 1877)
(226, 225, 77, u'medrecno', 1630)
(226, 225, 87, u'medrecno', 1043)
(226, 225, 87, u'medrecno', 1991)
(227, 225, 65, u'medrecno', 1749)
(228, 225, 78, u'medrecno', 1373)
(228, 225, 65, u'medrecno', 1290)
(226, 225, 2, u'medrecno', 1738)
(228, 225, 38, u'medrecno', 1686)
(226, 225, 30, u'medrecno', 1902)
(226, 225, 74, u'medrecno', 1730)
(226, 225, 62, u'medrecno', 1379)
(228, 225, 39, u'medrecno', 1459)
(228, 225, 77, u'medrecno', 1141)
(226, 225, 23, u'medrecno', 1939)
(228, 225, 96, u'medrecno', 1982)
(228, 225, 99, u'medrecno', 1863)
(228, 225, 87, u'medrecno', 1643)
(227, 225, 4, u'medrecno', 1788)
(227, 225, 69, u'medrecno', 1266)
(228, 225, 35, u'medrecno', 1290)
(226, 225, 87, u'medrecno', 1904)
(226, 225, 46, u'medrecno', 1629)
(228, 225, 77, u'medrecno', 1028)
(227, 225, 97, u

(226, 225, 27, u'medrecno', 1638)
(227, 225, 98, u'medrecno', 1074)
(228, 225, 4, u'medrecno', 1615)
(228, 225, 48, u'medrecno', 1664)
(228, 225, 5, u'medrecno', 1227)
(226, 225, 1, u'medrecno', 1800)
(227, 225, 40, u'medrecno', 1497)
(227, 225, 47, u'medrecno', 1640)
(228, 225, 84, u'medrecno', 1176)
(228, 225, 77, u'medrecno', 1867)
(228, 225, 12, u'medrecno', 1850)
(228, 225, 91, u'medrecno', 1649)
(228, 225, 40, u'medrecno', 1315)
(226, 225, 55, u'medrecno', 1946)
(227, 225, 16, u'medrecno', 1337)
(227, 225, 28, u'medrecno', 1157)
(228, 225, 68, u'medrecno', 1082)
(228, 225, 59, u'medrecno', 1635)
(228, 225, 47, u'medrecno', 1304)
(228, 225, 56, u'medrecno', 1851)
(227, 225, 65, u'medrecno', 1405)
(227, 225, 1, u'medrecno', 1026)
(227, 225, 83, u'medrecno', 1044)
(228, 225, 28, u'medrecno', 1953)
(228, 225, 65, u'medrecno', 1634)
(228, 225, 80, u'medrecno', 1317)
(227, 225, 99, u'medrecno', 1323)
(227, 225, 11, u'medrecno', 1295)
(228, 225, 43, u'medrecno', 1210)
(226, 225, 88, u'm

ProgrammingError: Cannot operate on a closed database.

The script below initially takes in the project_id used for the project. Once the input is read, the function named 
"create_view_sql" is used to spit out the query to construct the views by form names. The script below generates the queries only for non-checkbox fields (select, radio) and these are identified by reading in a text file named "Samp_dict_no_cb.txt". The text file contains a dictionary of form names as keys and associated field names as values. Then, the contents of text file are evaluated as a python expression. 

All the required columns to be present in the view are constructed with a list comprehension named reqlist which is then type casted into a string to ease query construction. The desired query to create the views is generated on running the script by form names for non-checkbox fields. 

The output obtained on running the script is a bunch of "create" view statements which should be executed on the SQLite environment to create the views.


In [ ]:
import sqlite3
import pandas as pd

import os


project_id = raw_input("REDCap Project ID#: ")

#Function that returns dictionary
def dict_read(readdict):

    return readdict

#Functon used to build query string
def view_text_string(vwtext):

    return vwtext

#function to create views
#Functions returns the SQL query string used for creating
#views based on form names. The queries generated are executed and the
#Resulting views are mapped on GeneTegra. They are easy to query as the
#views flip the REDCap data in EAV format  to a relational
#database which is more intuitive to query on and
#easier to query with GeneTegra
def create_view_sql(project_id):

    #This nested function is used to generate the joins
    #with all the field based attribute tables thus
    #resulting in a view that is based on form names
    #where in the form contains all the fields associated
    #with itself
    def cond_diff():
        countnumgen = 0
        for i in v:
            countnumgen += 1
            strct = str(countnumgen)
            if type(i) == str:

                joinstr = "\n LEFT JOIN table_" + str(project_id) + i + " t" + strct + \
                          "\n ON ((f.instance = " + i + "_instance) or " \
                           "(f.instance is null and " + i + "_instance is null)) \n " \
                            "and f.record = " + i + "_record \n "\
                            "and f.event_id = " + i + "_event_id"
                print joinstr

    #read csv into pandas daraframe
    reqpd = pd.read_csv(os.path.expanduser('~') + '\\sample_data_tables.csv')
    ps = reqpd['form_name'].tolist()


    #This text file is used to read a dictionary of both checkbox and non checkbox fields.
    #The argument to the dict_read() function neeeds to be changed based on
    #the dictionary being read based on checkbox/ non- checkbox fields.
    dictread = open(os.path.expanduser('~') + dict_read("\\Samp_dict_no_cb.txt"))

    empd = eval(dictread.read())

    #Typecasted to set
    setps = set(ps)

    view_text  = ""

    #Loop through the dictionary to fetch the desired fields and append
    #strings based on field names
    for k,v in empd.iteritems():

        reqlist =  [s + "," + s + "_event_id" + ","  + s + "_record" + "," + s + "_patient_id" + "," + s + "_label" + "," + s + "_instance" for s in v[1:]]
        reqstr = ",".join(reqlist)

        #The argument passed to the dict_read() function depends on
        #whether the checkbox/non-checkbox dictionary is being read.
        #Needs to be changed accordingly
        if dict_read("\\Samp_dict_no_cb.txt"):

            #The query string needed. Remove "_checkbox" for non checkbox fields
            view_text =  view_text_string("""\n\n\nDrop view if exists """) + k + ";" + "\n\n\nCREATE view """  + k + \
                                                         """ as SELECT distinct """ + reqstr + \
                     "\nFROM form_samp_join f "
            print view_text

            #Calling cond_diff() function to concatenate the joins with the "create view query"
            #To generate the desired query string
            cond_diff()
    

    return view_text

create_view_sql(225)


The script below is used to create the views by form names for only checkbox fields. It resembles the script above with the only distinction being the string "checkbox_" prepended to the query string. The dictionary read in contains the form names as keys and the checkbox-field names as values. The output obtained on running the script is a bunch of "create" view statements which should be executed on the SQLite environment to create the views.

In [ ]:
import sqlite3
import pandas as pd


import os


project_id = raw_input("REDCap Project ID#: ")

#Function that returns dictionary
def dict_read(readdict):

    return readdict

#Functon used to build query string
def view_text_string(vwtext):

    return vwtext

#function to create views
#Functions returns the SQL query string used for creating
#views based on form names. The queries generated are executed and the
#Resulting views are mapped on GeneTegra. They are easy to query as the
#views flip the REDCap data in EAV format  to a relational
#database which is more intuitive to query on and
#easier to query with GeneTegra
def create_view_sql(project_id):

    #This nested function is used to generate the joins
    #with all the field based attribute tables thus
    #resulting in a view that is based on form names
    #where in the form contains all the fields associated
    #with itself
    def cond_diff():
        countnumgen = 0
        for i in v:
            countnumgen += 1
            strct = str(countnumgen)
            if type(i) == str:

                joinstr = "\n LEFT JOIN checkbox_table_" + str(project_id) + i + " t" + strct + \
                          "\n ON ((f.instance = " + i + "_instance) or " \
                           "(f.instance is null and " + i + "_instance is null)) \n " \
                            "and f.record = " + i + "_record \n "\
                            "and f.event_id = " + i + "_event_id"
                print joinstr

    #read csv into pandas daraframe
    reqpd = pd.read_csv(os.path.expanduser('~') + '\\sample_data_tables.csv')
    ps = reqpd['form_name'].tolist()


    #This text file is used to read a dictionary of both checkbox and non checkbox fields.
    #The argument to the dict_read() function neeeds to be changed based on
    #the dictionary being read based on checkbox/ non- checkbox fields.
    dictread = open(os.path.expanduser('~') + dict_read("\\Samp_dict_checkboxes.txt"))

    empd = eval(dictread.read())

    #Typecasted to set
    setps = set(ps)

    view_text  = ""

    #Loop through the dictionary to fetch the desired fields and append
    #strings based on field names
    for k,v in empd.iteritems():

        reqlist =  [s + "," + s + "_event_id" + ","  + s + "_record" + "," + s + "_patient_id"  + "," + "Group_Concat(" + s +  "_code" +"," + "'" + "|" + "'" + ")" + "," + "Group_Concat(" + s +  "_label" +"," + "'" + "|" + "'" + ")" + "," + s + "_instance" for s in v[1:]]
        reqstr = ",".join(reqlist)
        lstgroupby = [s + "," + s + "_event_id" + ","  + s + "_record" + "," + s + "_patient_id"  + "," + s + "_instance" for s in v[1:]]
        strgroupby = ','.join(lstgroupby)
        
        
        #The argument passed to the dict_read() function depends on
        #whether the checkbox/non-checkbox dictionary is being read.
        #Needs to be changed accordingly
        if dict_read("\\Samp_dict_cb.txt"):

            #The query string needed. Remove "_checkbox" for non checkbox fields
            view_text =  view_text_string("""\n\n\nDrop view if exists """) + k + ";" + "\n\n\nCREATE view """  + k + \
                                                         """ as SELECT distinct """ + reqstr + \
                     "\nFROM form_samp_join f "
            print view_text

            #Calling cond_diff() function to concatenate the joins with the "create view query"
            #To generate the desired query string
            cond_diff()
            groupbystr = "Group by " + strgroupby + ';'
            print groupbystr
     
    

    return view_text

create_view_sql(225)
